#### Problem-solving

In [1]:
# Libraries
import pandas as pd 
import numpy as np
pd.options.mode.chained_assignment = None

In [2]:
# 1. Download the event_data.csv dataset, which contains data on the use of the mobile
# application of users who registered from July 29 to September 1, 2019
df = pd.read_csv("../files/event_data.csv")

In [3]:
# Preprocessing to later 
df['event_date'] = pd.to_datetime(df['event_date'])
df['event_date_str'] = df['event_date'].apply(lambda x: x.strftime("%Y-%m-%d"))
df['week_event'] = df['event_date'].apply(lambda x: x.strftime("%V"))

In [4]:
# 2. Highlight user cohorts based on the week of registration in the application. The cohort
# identifier should be the week ordinal (for example, the week from July 29 to August 4
# should have identifier 31).

print(f"Check if it's only one year or more, min: {df.event_date.min()} & max: {df.event_date.max()}")

map_registration = df['event_type'] == 'registration'
# Cross validation to check that there is no rows repeated
assert df[map_registration].user_id.nunique() ==  df[map_registration].shape[0], "There is a problem with the data"

# Get all info of registrations and put it int the original df
df_registration = df[map_registration]
df_registration['cohort_registration'] = df['event_date'].apply(lambda x: x.strftime("%V"))

# Final df for making analysis
df_processed = df.merge(df_registration[['user_id', 'cohort_registration']], on='user_id', how='inner')

Check if it's only one year or more, min: 2019-07-29 00:02:15 & max: 2019-09-01 23:59:54


In [5]:
df.shape, df_processed.shape

((79742, 6), (79742, 7))

In [6]:
# Some checks to be compliant with the requirements of the week format
cols_to_check = ['cohort_registration', 'event_date_str']

map_filter_1 = (df_processed['event_date_str']=='2019-07-29') & (df_processed['event_type'] == 'registration')
map_filter_2 = (df_processed['event_date_str']=='2019-08-04') & (df_processed['event_type'] == 'registration')
map_filter_3 = (df_processed['event_date_str']=='2019-08-05') & (df_processed['event_type'] == 'registration')
print(df_processed[map_filter_1].sample(1).iloc[0][cols_to_check])
print(df_processed[map_filter_2].sample(1).iloc[0][cols_to_check])
print(df_processed[map_filter_3].sample(1).iloc[0][cols_to_check])

cohort_registration            31
event_date_str         2019-07-29
Name: 353, dtype: object
cohort_registration            31
event_date_str         2019-08-04
Name: 20819, dtype: object
cohort_registration            32
event_date_str         2019-08-05
Name: 21165, dtype: object


In [7]:
# 3. How many unique users in the cohort with ID 33?
map_week_33 = df_processed['cohort_registration'] == '33'
print("A total of unique user of week 33:", df_processed[map_week_33].user_id.nunique())

A total of unique user of week 33: 2045


In [8]:
# 4. For each event, highlight the indicator lifetime - the weekly lifetime of the cohort. The
# lifetime indicator is calculated based on the serial number of the week in which the event
# is committed, relative to the week of registration. For example, an event committed on
# August 3 by a user from a cohort of registrants at 31 weeks will be committed on the zero
# week of lifetime, and an event committed by the same user on August 5 will be committed
# on the first week of lifetime).

# Check data
#map_filter = (df_processed['event_date_str']=='2019-08-03') & (df_processed['event_type'] == 'registration')
#print(df_processed[map_filter].sample(1).iloc[0][cols_to_check])

df_processed['life_time'] = df_processed['week_event'].astype(int) - df_processed['cohort_registration'].astype(int) 
df_processed.sample(5)

,user_id,event_date,event_type,purchase_amount,event_date_str,week_event,cohort_registration,life_time
45355,187639,2019-08-13 06:45:08,registration,NaN,2019-08-13,33,33,0
26193,c2a273,2019-08-12 05:31:27,simple_event,NaN,2019-08-12,33,32,1
13364,a14750,2019-08-14 20:10:08,simple_event,NaN,2019-08-14,33,31,2
79386,ecb889,2019-09-01 02:55:31,simple_event,NaN,2019-09-01,35,35,0
38832,857fe7,2019-08-22 17:09:39,simple_event,NaN,2019-08-22,34,32,2


In [18]:
# 5. Build a summary table of changes in the Retention Rate for cohorts depending on lifetime.
value_target = 'user_id'
df_total = df_processed.groupby(['cohort_registration', 'life_time'])[value_target].apply(lambda x: x.nunique()).reset_index()
df_pivot = df_total.pivot_table(index='cohort_registration', columns='life_time', values=value_target)
df_pivot = df_pivot.fillna(0)
retention_df = df_pivot.divide(df_pivot.iloc[:, 0], axis=0)
retention_df

life_time,0,1,2,3,4
cohort_registration,,,,,
31,1.0,0.927595,0.629367,0.356962,0.15038
32,1.0,0.929303,0.648053,0.361168,0.00000
33,1.0,0.924205,0.661125,0.000000,0.00000
34,1.0,0.929078,0.000000,0.000000,0.00000
35,1.0,0.000000,0.000000,0.000000,0.00000


In [10]:
# 6. What is the 3 week retention rate for a cohort with ID 32? Give the answer in percent,
# rounded to 2 decimal places, inclusive.

# .loc[cohort_retention][life_time]
value = retention_df.loc['32'][3]
print(f"The retention rate for cohort 32 on week 3 is {round(value*100,2)}%")

The retention rate for cohort 32 on week 3 is 36.12%


In [11]:
# 7. Build a summary table of changes in the indicator ARPPU (Average Revenue Per Paying
# User) for cohorts depending on lifetime


# ARPPU is based in paying customers only.

map_paying_users =  ~df_processed['purchase_amount'].isnull()
df_total_arppu = df_processed[map_paying_users].groupby(['cohort_registration', 'life_time']).agg({"purchase_amount": "sum", 
                                                                                                   "user_id": lambda x: x.nunique()}).reset_index()

df_total_arppu['arppu_indicator'] = df_total_arppu['purchase_amount'] / df_total_arppu['user_id']
df_pivot_arppu = df_total_arppu.pivot_table(index='cohort_registration', columns='life_time', values='arppu_indicator')

In [12]:
# Original table
df_pivot_arppu

life_time,0,1,2,3,4
cohort_registration,,,,,
31,30.445205,35.413793,35.911765,33.081081,31.40625
32,32.978723,35.024958,35.294118,33.297872,NaN
33,33.186441,35.858086,34.784810,NaN,NaN
34,33.843416,35.904936,NaN,NaN,NaN
35,34.094955,NaN,NaN,NaN,NaN


In [13]:
# 8. What is the 3-week ARPPU of a cohort with ID 31? Give the answer with a floating point
# number, rounded to 2 decimal places, inclusive.

print(f"The 3-week ARPPU of a cohort with ID 31 is {round(df_pivot_arppu.loc['31'][3],2)}")

The 3-week ARPPU of a cohort with ID 31 is 33.08


In [14]:
# 9. What is the median time between user registration and first purchase? Give the answer
# in seconds (!) As an integer.
map_base_users = df_processed['event_type'].isin(['purchase', 'registration'])
base_users = df_processed[map_base_users]

# Order the dataframe, first purchase at the top
base_users.sort_values(by=['user_id', 'event_date'], ascending=True, inplace=True)
base_processed = base_users.drop_duplicates(subset=['user_id', 'event_type'])

# users with purchases
valid_users = base_processed.groupby('user_id').event_type.count().reset_index()
base_processed = base_processed[base_processed['user_id'].isin(valid_users[valid_users['event_type']>1]['user_id'])]



In [15]:
base_processed['time_diff_raw'] = base_processed['event_date'].shift(-1) - base_processed['event_date']
base_processed['time_diff'] = base_processed['time_diff_raw'].apply(lambda x: x.total_seconds())

map_base_diff = base_processed['event_type'] == 'registration'
print(f"The median time in seconds between the registration and the first purchase is\
 {int(base_processed[map_base_diff]['time_diff'].median())}s")

The median time in seconds between the registration and the first purchase is 434774s


In [16]:
# Curiosity 
base_processed[map_base_diff]['time_diff'].describe()

count    4.283000e+03
mean     5.416124e+05
std      4.256284e+05
min      6.600000e+01
25%      2.232475e+05
50%      4.347740e+05
75%      7.490150e+05
max      2.539779e+06
Name: time_diff, dtype: float64